In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
import rfpy
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada) ;  arquivos

(#3) [Path('C:/Users/rsilva/Downloads/entrada/201201_T153421_OneThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/201201_T154509_MultiplesThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/rfeye002304_SLMA_bimestral_PEAK_200829_234902.bin')]

In [ ]:
one_tid = arquivos[-1]
mult_tid = arquivos[1]

On a normal conda environment the time of the following cell was
```python
%%timeit
one = process_bin(one_tid)
```
1.22 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %%timeit
# one = parse_bin(one_tid)

On a normal conda environment the time of the following cell was
```python
%%timeit
multi = process_bin(mult_tid)
```
1.29 s ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %%timeit
# multi = parse_bin(mult_tid)

In [ ]:
one = parse_bin(one_tid)

In [ ]:
one.keys()

dict_keys(['file_version', 'string', 'blocos'])

In [ ]:
one['blocos']

defaultdict(fastcore.foundation.L,
            {(21,
              0): (#1) [<rfpy.blocks.DType21 object at 0x0000026560EF8F08>],
             (41,
              0): (#2) [<rfpy.blocks.DType41 object at 0x0000026560EF8D48>,<rfpy.blocks.DType41 object at 0x0000026560EF8348>],
             (24,
              1): (#1) [<rfpy.blocks.DType24 object at 0x0000026560EF8708>],
             (24,
              5): (#6) [<rfpy.blocks.DType24 object at 0x0000026560EF8308>,<rfpy.blocks.DType24 object at 0x0000026560EF87C8>,<rfpy.blocks.DType24 object at 0x0000026560EF8208>,<rfpy.blocks.DType24 object at 0x0000026560EF81C8>,<rfpy.blocks.DType24 object at 0x0000026560EF8188>,<rfpy.blocks.DType24 object at 0x0000026560EF8108>],
             (24,
              20): (#1) [<rfpy.blocks.DType24 object at 0x0000026560F059C8>],
             (40,
              1): (#6605) [<rfpy.blocks.DType40 object at 0x0000026560F05A48>,<rfpy.blocks.DType40 object at 0x0000026560F05E88>,<rfpy.blocks.DType40 object at 0x000

In [ ]:
multi = parse_bin(mult_tid)

In [ ]:
multi.keys()

dict_keys(['file_version', 'string', 'blocos'])

In [ ]:
multi['file_version']

22

In [ ]:
multi['string']

'CRFS DATA FILE V022'

In [ ]:
multi['blocos']

defaultdict(fastcore.foundation.L,
            {(21,
              0): (#1) [<rfpy.blocks.DType21 object at 0x0000026560CD5E48>],
             (41,
              0): (#2) [<rfpy.blocks.DType41 object at 0x0000026560CDB248>,<rfpy.blocks.DType41 object at 0x0000026560CDB208>],
             (24,
              5): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD9888>],
             (24,
              10): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD9F48>],
             (24,
              20): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD9848>],
             (24,
              30): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD9988>],
             (24,
              40): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD97C8>],
             (24,
              50): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD9E88>],
             (24,
              60): (#1) [<rfpy.blocks.DType24 object at 0x0000026560CD91C8>],
             (24,
              70): (#1) [<rfpy.bloc

In [ ]:
x = multi['blocos'][(63,40)][0]

In [ ]:
getattrs(x)

{'agc_array': '30-34-39-48-48-46-42',
 'bw': 40,
 'data_points': 2048,
 'datetime_stamp': datetime.datetime(2020, 12, 1, 15, 45, 9, 816463),
 'global_error_code': -1,
 'global_flags_code': -1,
 'group_id': 0,
 'id_antenna': 0,
 'level_offset': -20,
 'n_agc': 7,
 'n_padding': -20,
 'n_tunning': 7,
 'num_meas': 1,
 'passo': 0.019540791402051783,
 'processing': 'average',
 'rbw': 19540,
 'size': 2136,
 'spent_time_microsecs': 5570,
 'start_channel': 0,
 'start_mega': 70,
 'start_mili': 0,
 'stop_channel': 0,
 'stop_mega': 110,
 'stop_mili': 0,
 'thread_id': 40,
 'tunning_info': 'completed without error',
 'type': 63,
 'unit': '%'}

In [ ]:
%%time
b1 = multi['blocos'][(63,40)].map(getattrs)

Wall time: 500 ms


In [ ]:
df = pd.DataFrame(b1).set_index('datetime_stamp') ; df.head()

,agc_array,bw,data_points,global_error_code,global_flags_code,group_id,id_antenna,level_offset,n_agc,n_padding,...,start_channel,start_mega,start_mili,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit
datetime_stamp,,,,,,,,,,,,,,,,,,,,,
2020-12-01 15:45:09.816463,30-34-39-48-48-46-42,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,40,completed without error,63,%
2020-12-01 15:45:10.318062,30-34-38-46-50-46-41,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,40,completed without error,63,%
2020-12-01 15:45:10.817929,30-32-39-48-50-46-40,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,40,completed without error,63,%
2020-12-01 15:45:11.316203,30-32-36-46-48-46-41,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,40,completed without error,63,%
2020-12-01 15:45:11.810011,30-32-37-46-48-46-40,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,40,completed without error,63,%


In [ ]:
b2 = multi['blocos'][(63,50)].map(getattrs)

In [ ]:
df2 = pd.DataFrame(b2).set_index('datetime_stamp') ; df2.head()

,agc_array,bw,data_points,global_error_code,global_flags_code,group_id,id_antenna,level_offset,n_agc,n_padding,...,start_channel,start_mega,start_mili,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit
datetime_stamp,,,,,,,,,,,,,,,,,,,,,
2020-12-01 15:45:13.817044,32-34-39-48-50-48-42,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,50,completed without error,63,%
2020-12-01 15:45:18.812037,32-34-40-48-50-48-42,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,50,completed without error,63,%
2020-12-01 15:45:23.818756,32-34-39-48-50-48-43,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,50,completed without error,63,%
2020-12-01 15:45:28.842880,32-34-39-48-50-48-42,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,50,completed without error,63,%
2020-12-01 15:45:33.842036,32-34-39-48-50-48-42,40,2048,-1,-1,0,0,-20,7,-20,...,0,70,0,0,110,0,50,completed without error,63,%


In [ ]:
b3 = one['blocos'][(63,20)].map(getattrs)

In [ ]:
df3 = pd.DataFrame(b3).set_index('datetime_stamp') ; df3.head()

,agc_array,bw,data_points,global_error_code,global_flags_code,group_id,id_antenna,level_offset,n_agc,n_padding,...,start_channel,start_mega,start_mili,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit
datetime_stamp,,,,,,,,,,,,,,,,,,,,,
2020-08-29 23:50:01.059268,44-44-42-42-42-40-38-38-37-36-36-36-36-36-38-3...,29,14848,-1,-1,0,0,-20,32,-20,...,0,108,0,0,137,0,20,completed without error,63,%
2020-08-29 23:51:00.579360,44-44-42-42-40-37-37-37-38-37-36-36-36-36-38-3...,29,14848,-1,-1,0,0,-20,32,-20,...,0,108,0,0,137,0,20,completed without error,63,%
2020-08-29 23:52:01.969255,44-42-42-42-40-38-38-36-37-36-36-36-36-36-38-3...,29,14848,-1,-1,0,0,-20,32,-20,...,0,108,0,0,137,0,20,completed without error,63,%
2020-08-29 23:53:01.359546,44-44-42-42-40-40-37-37-38-36-36-36-36-36-38-3...,29,14848,-1,-1,0,0,-20,32,-20,...,0,108,0,0,137,0,20,completed without error,63,%
2020-08-29 23:54:00.959245,44-44-42-42-40-40-37-37-37-37-36-37-36-36-40-3...,29,14848,-1,-1,0,0,-20,32,-20,...,0,108,0,0,137,0,20,completed without error,63,%


In [ ]:
dfs = [pd.DataFrame(v.map(getattrs)) for v in one['blocos'].values() ]

In [ ]:
dfs[0]

,hostname,method,size,thread_id,type,unit_info
0,rfeye002304,CRFS default method,60,0,21,   RFeye002304


In [ ]:
dfs[1]

,identifier,len_text,size,text,thread_id,type,wallclock_date_time_stamp
0,LOGGER_NAME,12,60,log_main,0,41,"((29, 8, 20, 0), <bound method DType41.time of..."
1,LOGGER_VERSION,64,112,build_version=1.6.7;build_date=Oct 2 2018;bui...,0,41,"((29, 8, 20, 0), <bound method DType41.time of..."


In [ ]:
dfs[2]

,group_id,len_text,size,text,thread_id,type
0,0,12,20,GPS Data,1,24


In [ ]:
dfs[3]

,group_id,len_text,size,text,thread_id,type
0,0,40,48,Loaded mask for %(PMEC_bimestral)s-on-,5,24
1,0,72,80,PMEC: Unit %(unit)s executando medidas do PMEC...,5,24
2,0,92,100,PMEC: Unit %(unit)s configurada para nao execu...,5,24
3,0,32,40,Loaded mask for %(PMEC)s-on-,5,24
4,0,48,56,PMEC: Unit %(unit)s executando medidas do PMEC.,5,24
5,0,68,76,PMEC: Unit %(unit)s configurada para nao execu...,5,24


In [ ]:
dfs[4]

,group_id,len_text,size,text,thread_id,type
0,0,32,40,Pico medio em 1 minuto SLMA2,20,24


In [ ]:
dfs[5]

,altitude,gps_date,gps_date_time_stamp,gps_status,gps_time,heading,latitude,longitude,num_sattelites,size,speed,thread_id,type,wallclock_date_time_stamp
0,619.6,"(29, 8, 20, 0)","((29, 8, 20, 0), (23, 50, 0, 0))",Differential GPS,"(23, 50, 0, 0)",0.0,-21.228926,-47.759933,11,40,0.016,1,40,"((29, 8, 20, 0), (23, 50, 0, 0), 698607000)"
1,615.6,"(29, 8, 20, 0)","((29, 8, 20, 0), (23, 51, 0, 0))",Differential GPS,"(23, 51, 0, 0)",0.0,-21.228928,-47.759912,11,40,0.031,1,40,"((29, 8, 20, 0), (23, 51, 0, 0), 221746000)"
2,612.0,"(29, 8, 20, 0)","((29, 8, 20, 0), (23, 52, 1, 0))",Differential GPS,"(23, 52, 1, 0)",0.0,-21.228921,-47.759922,10,40,0.014,1,40,"((29, 8, 20, 0), (23, 52, 1, 0), 599649000)"
3,617.2,"(29, 8, 20, 0)","((29, 8, 20, 0), (23, 53, 0, 0))",Differential GPS,"(23, 53, 0, 0)",0.0,-21.228934,-47.759922,10,40,0.012,1,40,"((29, 8, 20, 0), (23, 53, 0, 0), 989603000)"
4,624.0,"(29, 8, 20, 0)","((29, 8, 20, 0), (23, 54, 0, 0))",Differential GPS,"(23, 54, 0, 0)",0.0,-21.228931,-47.759926,10,40,0.007,1,40,"((29, 8, 20, 0), (23, 54, 0, 0), 597631000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,620.5,"(3, 9, 20, 0)","((3, 9, 20, 0), (13, 50, 1, 0))",Differential GPS,"(13, 50, 1, 0)",0.0,-21.228957,-47.759923,11,40,0.018,1,40,"((3, 9, 20, 0), (13, 50, 1, 0), 532466000)"
6601,621.1,"(3, 9, 20, 0)","((3, 9, 20, 0), (13, 51, 0, 0))",Differential GPS,"(13, 51, 0, 0)",0.0,-21.228941,-47.759911,11,40,0.033,1,40,"((3, 9, 20, 0), (13, 51, 0, 0), 450485000)"
6602,619.1,"(3, 9, 20, 0)","((3, 9, 20, 0), (13, 52, 1, 0))",Differential GPS,"(13, 52, 1, 0)",0.0,-21.228968,-47.759907,11,40,0.029,1,40,"((3, 9, 20, 0), (13, 52, 1, 0), 140490000)"
6603,620.3,"(3, 9, 20, 0)","((3, 9, 20, 0), (13, 53, 0, 0))",Differential GPS,"(13, 53, 0, 0)",0.0,-21.228954,-47.759906,11,40,0.025,1,40,"((3, 9, 20, 0), (13, 53, 0, 0), 52433000)"


In [ ]:
dfs[6]

,agc_array,bw,data_points,datetime_stamp,global_error_code,global_flags_code,group_id,id_antenna,level_offset,n_agc,...,start_channel,start_mega,start_mili,stop_channel,stop_mega,stop_mili,thread_id,tunning_info,type,unit
0,44-44-42-42-42-40-38-38-37-36-36-36-36-36-38-3...,29,14848,2020-08-29 23:50:01.059268,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
1,44-44-42-42-40-37-37-37-38-37-36-36-36-36-38-3...,29,14848,2020-08-29 23:51:00.579360,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
2,44-42-42-42-40-38-38-36-37-36-36-36-36-36-38-3...,29,14848,2020-08-29 23:52:01.969255,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
3,44-44-42-42-40-40-37-37-38-36-36-36-36-36-38-3...,29,14848,2020-08-29 23:53:01.359546,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
4,44-44-42-42-40-40-37-37-37-37-36-37-36-36-40-3...,29,14848,2020-08-29 23:54:00.959245,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,44-44-42-42-40-38-37-37-37-36-36-36-36-36-38-3...,29,14848,2020-09-03 13:50:01.891165,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
6601,44-44-42-42-40-38-37-38-37-36-36-36-36-36-38-3...,29,14848,2020-09-03 13:51:00.811174,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
6602,44-44-42-42-40-38-37-38-36-36-36-36-36-36-38-3...,29,14848,2020-09-03 13:52:01.501244,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%
6603,44-42-42-40-40-38-37-38-38-36-36-36-36-36-38-3...,29,14848,2020-09-03 13:53:00.411253,-1,-1,0,0,-20,32,...,0,108,0,0,137,0,20,completed without error,63,%


In [ ]:
dfs[7]

IndexError: list index out of range

In [ ]:
dfs[8]

,group_id,len_text,size,text,thread_id,type
0,0,24,32,trace mode: Clear/Write,60,24


In [ ]:
dfs[9]

,group_id,len_text,size,text,thread_id,type
0,0,32,40,"trace mode: MaxHold, count=2",70,24


In [ ]:
dfs[10]

,group_id,len_text,size,text,thread_id,type
0,0,32,40,"trace mode: MaxHold, count=5",80,24


In [ ]:
dfs[11]

,group_id,len_text,size,text,thread_id,type
0,0,32,40,"trace mode: MaxHold, count=10",90,24
